In [1]:
from ethpandaops_python.preprocessor import Preprocessor
from eip4844_blob_data.panel_charts import create_slot_inclusion_line_chart, create_priority_fee_chart, get_slot_inclusion_table
from eip4844_blob_data.polars_preprocess import create_slot_inclusion_df, create_slot_gas_bidding_df
from holoviews import opts
import nest_asyncio
import polars as pl
import panel as pn
import time


nest_asyncio.apply()
pn.extension("tabulator", template="material", sizing_mode="stretch_width")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Data Query

In [2]:
# labeled blobs - https://dune.com/queries/3521610
sequencers_l2: dict[list[str], list[str]] = {
    "sequencer_addresses": [
        # should be the "from" addresses, this is what hilldobby SQL query does.
        # should also be proper checksum, not lowercase
        "0xC1b634853Cb333D3aD8663715b08f41A3Aec47cc",
        "0x5050F69a9786F081509234F1a7F4684b5E5b76C9",
        "0x6887246668a3b87F54DeB3b94Ba47a6f63F32985",
        "0x000000633b68f5D8D3a86593ebB815b4663BCBe0",
        "0x415c8893D514F9BC5211d36eEDA4183226b84AA7",
        "0xa9268341831eFa4937537bc3e9EB36DbecE83C7e",
        "0xcF2898225ED05Be911D3709d9417e86E0b4Cfc8f",
        "0x0D3250c3D5FAcb74Ac15834096397a3Ef790ec99",
        "0xC70ae19B5FeAA5c19f576e621d2bad9771864fe2",
        "0xC94C243f8fb37223F3EB2f7961F7072602A51B8B"
    ],
    "sequencer_names": [
        "arbitrum",
        "base",
        "optimism",
        "taiko",
        "blast",
        "linea",
        "scroll",
        "zksync",
        "paradex",
        "metal"
    ],
}

num_days: int = 7

In [3]:
def get_data() -> dict[str, pl.DataFrame]:
    return Preprocessor(
        blob_producer=sequencers_l2,
        period=num_days,
        network="mainnet",  # mainnet
    ).cached_data

In [4]:
def filter_data_seq(
    sequencers: list[str], slot_inclusion_joined_df: pl.DataFrame, cached_data: dict[str, pl.DataFrame]
) -> dict[str: pl.DataFrame]:
    """
    This function filters a dataframe and returns updated chart data, based on the input of the dashboard user.
    """

    # slot inclusion
    slot_inclusion_df = (
        slot_inclusion_joined_df.filter(
            pl.col("sequencer_names").is_in(sequencers))
        .filter(pl.col('meta_network_name') == 'mainnet')
        .unique()
        .sort(by="slot")
    )

    # gas bidding scatterplot median
    slot_gas_groupby_df = (
        slot_inclusion_df.group_by("slot_inclusion_rate", "sequencer_names")
        .agg(
            pl.col("priority_fee_bid_percent_premium").median(),
            pl.col("base_fee_per_gas").mean(),
        )
        .sort(by="slot_inclusion_rate")
        .drop_nulls()
    )

    return {
        # time series
        "slot_inclusion_df": slot_inclusion_df,
        "slot_gas_groupby_df": slot_gas_groupby_df,
    }

In [5]:
# start dashboard
def start_interactive_panel(filtered_data_dict, sequencer_names_list):
    multi_select = pn.widgets.MultiSelect(
        name="Sequencers",
        size=10,
        options=sequencer_names_list,
        value=sequencer_names_list,
    )

    # initial chart and table data
    slot_inclusion_line_chart = create_slot_inclusion_line_chart(
        filtered_data_dict["slot_inclusion_df"], sequencer_names_list
    )

    priority_fee_chart = create_priority_fee_chart(
        # add in filter to remove outliers and make chart look better
        filtered_data_dict["slot_inclusion_df"].filter(
            pl.col('slot_inclusion_rate') < 50),
        filtered_data_dict["slot_gas_groupby_df"].filter(
            pl.col('slot_inclusion_rate') < 50),
        sequencer_names_list,
    )

    # 6/9/24 DONT USE FOR NOW. I THINK I WILL REPLACE WITH DIFFERNT CHART?
    # avg_slot_inclusion_scatterplot = (filtered_data_dict['slot_inclusion_df'].group_by(
    #     'sequencer_names').agg(
    #     pl.col('fill_percentage').mean().alias('avg_fill_percentage'),
    #     pl.col('submission_count').mean().alias(
    #         'avg_submission_count'),
    #     pl.col('slot_inclusion_rate').mean().round(
    #         3).alias('avg_slot_inclusion_rate'),
    #     pl.col('blob_hashes_length').mean().alias(
    #         'avg_blob_hashes_length'),
    #     pl.len().alias('tx_count'),
    # ).sort(by='avg_slot_inclusion_rate')
    #     .plot.scatter(x='avg_slot_inclusion_rate', y='avg_blob_hashes_length', by='sequencer_names', width=800,
    #                   height=375, xlabel='Avg Slot Inclusion Rate', ylabel='Blob Tx Size', title='Avg Slot Inclusion Rate')
    # )

    sequencer_macro_blob_table: pl.DataFrame = (
        filtered_data_dict['slot_inclusion_df'].drop_nulls().unique().group_by(
            'sequencer_names').agg(
            pl.col('fill_percentage').mean().alias('avg_fill_percentage'),
            pl.col('submission_count').mean().alias(
                'avg_submission_count'),
            pl.col('slot_inclusion_rate').mean().round(
                3).alias('avg_slot_inclusion_rate'),
            pl.col('blob_hashes_length').mean().alias(
                'avg_blob_hashes_length'),
            pl.len().alias('tx_count'),
            pl.col('blob_hashes_length').sum().alias('blob_count'),
            pl.col('base_tx_fee_eth').sum().round(
                3).alias('total_base_fees_eth'),
            pl.col('priority_tx_fee_eth').sum().round(
                3).alias('total_priority_fees_eth'),
            pl.col('total_tx_fee_eth').sum().round(3).alias('total_eth_fees'),
            pl.col('priority_fee_gas').mean().round(
                3).alias('avg_priority_fee_bid'),
        ).rename({'sequencer_names': 'rollup', 'avg_blob_hashes_length': 'avg_blobs_in_tx'}))

    slot_inclusion_table_tabulator = get_slot_inclusion_table(
        filtered_data_dict["slot_inclusion_df"], sequencer_names_list)

    filename, button = slot_inclusion_table_tabulator.download_menu(
        text_kwargs={'name': 'Enter filename', 'value': 'default.csv'},
        button_kwargs={'name': 'Download data'},
    )

    entire_panel = pn.Column(
        pn.Row(
            pn.pane.Markdown(
                """
            # EIP-4844 Slot Inclusion Dashboard

            ## About
            This dashboard shows detailed analytics for blob inclusion rates as well as the efficiency of using EIP-1559 priority fees
            as a bidding mechanism for faster slot inclusion. This dashboard is made using [Xatu Data](https://github.com/ethpandaops/xatu-data?tab=readme-ov-file) for EL mempool and Beacon chain data and [Hypersync](https://github.com/enviodev/hypersync-client-python) 
            for transaction gas data for the [EIP-4844 data challenge](https://esp.ethereum.foundation/data-challenge-4844).
            """
            ),
            multi_select,
            styles=dict(background="WhiteSmoke"),
        ),
        pn.pane.Markdown(
            """
            ## 7 Day Historical Slot Inclusion
            When a transaction is resubmitted with updated gas parameters, the transaction hash changes. For example take this blob reference hash - 0x01c738cf37c911334c771f2295c060e5bd7d084f347e4334863336724934c59a. 
            On [etherscan](https://etherscan.io/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69) we can see that the transaction was replaced by the user. We can see on Ethernow that the transaction contains 
            the same blob reference hash in both the [original tx](https://www.ethernow.xyz/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69?batchIndex=1) and the [resubmitted tx](https://www.ethernow.xyz/tx/0x5a4094662bd05ff3639a8979927ab527e007a6925387951a9c1b3d2958b13a86?batchIndex=1).
            
            We can measure the total time that a blob hash sat in the mempool by subtracting the original tx was first seen from the slot time, when it eventually is finalized on the beacon chain. 
            In this particular example, the total time that the blob sat in the mempool was not from 18:56:27 to 18:57:11 (4 slots), but really 18:54:29 to 18:57:11 (14 slots)
            """
        ),
        pn.Row(
            slot_inclusion_line_chart.opts(axiswise=True),
            priority_fee_chart.opts(legend_position="left", show_legend=True),
            styles=dict(background="WhiteSmoke"),

        ),
        pn.Row(
            pn.pane.Markdown(
                """
            ## Slot Inclusion Rates
            **Slot Inclusion Rate** - The slot inclusion rate indicates the number of slots required for a blob to be included in the beacon chain, 
            with a higher rate signifying a longer inclusion time. The accompanying time-series chart tracks this metric from initial mempool 
            appearance to final beacon block inclusion. A 50 blob slot inclusion average is taken to smooth out the performance. 
            The target slot inclusion rate is 2. 
                """
            ),
            pn.pane.Markdown(
                """
            ## EIP-1559 Priority Fee Premium Correlation with Slot Rates
            The scatterplot illustrates the relationship between the EIP-1559 priority fee bid premiums and slot inclusion rates. The scatterplot points
            are individual blob bid datapoints and the line is a median bid premium. A higher priority fee bid premium tends to coincide 
            with longer slot inclusion times. This unexpected twist underscores the value of efficient slot utilization. The data indicates a trend 
            where higher bid premiums are associated with longer slot inclusion times, suggesting that as the time for a blob to be included 
            in the beacon chain increases, so does the priority fee bid premium. This behavior comes from the fact that if a blob sits in the 
            mempool for too long, then it is resubmitted with a higher priority fee. 
            """
            ),
            styles=dict(background="WhiteSmoke")
        ),
        pn.Row(
            pn.pane.Markdown(
                """
                # Blob Transaction Data (Past 7 days):
                
                This table provides detailed information on various metrics related to traditional transaction hashes that carry blob hashes. The metrics include:

                Blob Fill Percentage: Indicates the percentage of the transaction space filled by blobs.
                Transaction Resubmission Count: The number of times a transaction has been resubmitted with the same blobs.
                Number of Blobs in a Transaction: The count of blobs contained within a single transaction.
                ETH Priority Fees: The priority fees associated with each transaction in ETH.
                Additional Metrics: Various other relevant metrics, such as fees and timings.
                """
            ),
            styles=dict(background="WhiteSmoke"),
        ),
        pn.Row(
            pn.widgets.Tabulator(
                sequencer_macro_blob_table.to_pandas(), layout='fit_data'
            ),
            # avg_slot_inclusion_scatterplot,
            styles=dict(background="WhiteSmoke")
        ),
        pn.Column(
            pn.pane.Markdown(
                """
                # Slot Inclusion Data Table
                The table shows raw data that the dashboard was built on
                """
            ),
            pn.Column(filename, button),
            slot_inclusion_table_tabulator,
            styles=dict(background="WhiteSmoke")
        )
    )

    def update_bar_chart(event):
        """
        Use this to update charts based on sequencer name user selection
        """
        entire_panel[2][0].object = create_slot_inclusion_line_chart(
            filtered_data_dict["slot_inclusion_df"],
            sequencers=multi_select.value,
        )

        entire_panel[2][1].object = create_priority_fee_chart(
            filtered_data_dict["slot_inclusion_df"],
            filtered_data_dict["slot_gas_groupby_df"],
            sequencers=multi_select.value,
        )

    multi_select.param.watch(update_bar_chart, "value")

    return entire_panel

### What charts to add in?

In [6]:
# # retrieve data from database and store in memory
cached_data = get_data()

# prepare dataframes
slot_inclusion_df = create_slot_inclusion_df(
    cached_data).join(
    pl.from_dict(sequencers_l2),
    left_on="from",
    right_on="sequencer_addresses",
    how="left",
    coalesce=True
).select('slot', 'slot_time', 'hash', 'fill_percentage', 'submission_count', 'slot_inclusion_rate', 'sequencer_names', 'meta_network_name')

slot_gas_bidding_df = create_slot_gas_bidding_df(
    cached_data).select('block_number', 'hash', 'base_tx_fee_eth', 'priority_tx_fee_eth', "base_fee_per_gas",
                        "priority_fee_gas", 'total_tx_fee_eth', 'priority_fee_bid_percent_premium')

slot_inclusion_joined_df = slot_inclusion_df.join(
    slot_gas_bidding_df, on="hash", how="left", coalesce=True
)

2024-06-09 is within a day of 2024-06-09


In [7]:
slot_inclusion_joined_df.filter(pl.col('sequencer_names') == 'taiko').sort(
    by='priority_tx_fee_eth', descending=True).drop_nulls()

slot,slot_time,hash,fill_percentage,submission_count,slot_inclusion_rate,sequencer_names,meta_network_name,block_number,base_tx_fee_eth,priority_tx_fee_eth,base_fee_per_gas,priority_fee_gas,total_tx_fee_eth,priority_fee_bid_percent_premium
u32,datetime[ms],str,f64,u32,f64,str,str,u64,f64,f64,f64,f64,f64,f64
9230845,2024-06-05 13:29:23,"""0x829e82526f2ce8dc3f8294872cdcbd89ede7a7eb528fae59a4a4a900714f6069""",97.47000000000001,1,5,"""taiko""","""mainnet""",20025932,0.01472578581183566,0.00682122,43.176398978,20,0.02154700581183566,0.317
9235223,2024-06-06 04:04:59,"""0x73b13836480caf45033bd15ecee228252cdc73d0c7b68771ffdf0f9f32a1fcf6""",47.12,1,5,"""taiko""","""mainnet""",20030287,0.003258922736623892,0.006651740000000001,9.798707516,20,0.009910662736623893,0.671
9231274,2024-06-05 14:55:11,"""0x43095739f6a219e21b4161cebbd232f44c0d06ebc88d8f73d8bda8d212b2fce1""",97.59999999999998,1,2,"""taiko""","""mainnet""",20026358,0.012770053525080606,0.006299310000000001,20.272146513000003,10,0.019069363525080606,0.33
9231279,2024-06-05 14:56:11,"""0x35e3a16839690b9b5272f837ac09297221f02761991d531bf343ce690b4f8978""",97.34999999999998,1,1,"""taiko""","""mainnet""",20026363,0.012690035943260947,0.006299310000000001,20.145120566000003,10,0.018989345943260946,0.332
9235810,2024-06-06 06:02:23,"""0x17863f6c92060f8fe449b6fc847b01b8610a00545aad4e3e12e0ab7929450b74""",14.36,1,1,"""taiko""","""mainnet""",20030869,0.006113324652435006,0.006299310000000001,9.704752826,10,0.012412634652435008,0.507
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9246432,2024-06-07 17:26:47,"""0x04c7d72ea6a0a6c412401a24e00bfb78c4aaddf024ebde9f10228bfbeb4f6c6e""",9.45,1,1,"""taiko""","""mainnet""",20041434,0.004585998449697245,0.000026477900000000003,17.320098836,0.1,0.004612476349697245,0.006
9245928,2024-06-07 15:45:59,"""0xe31adb0574f49f32eb04b8f70215edead417fb5a759dab660bdb0b647e2dd32f""",6.18,1,3,"""taiko""","""mainnet""",20040932,0.006262900692297676,0.000026476700000000004,23.654385525000002,0.1,0.006289377392297676,0.004
9246023,2024-06-07 16:04:59,"""0x997e41bfb60706f19339bb54af16154c5a2062d3e20d88a49da326d1ba583580""",1.8500000000000003,1,1,"""taiko""","""mainnet""",20041027,0.005563209533086825,0.000026476700000000004,21.011717975,0.1,0.005589686233086825,0.005


In [8]:
slot_inclusion_joined_df.filter(pl.col('sequencer_names') == 'taiko').unique().with_columns(
    pl.col('total_tx_fee_eth').sum().alias('sum'),
    pl.col('total_tx_fee_eth').mean().alias('mean')
)

slot,slot_time,hash,fill_percentage,submission_count,slot_inclusion_rate,sequencer_names,meta_network_name,block_number,base_tx_fee_eth,priority_tx_fee_eth,base_fee_per_gas,priority_fee_gas,total_tx_fee_eth,priority_fee_bid_percent_premium,sum,mean
u32,datetime[ms],str,f64,u32,f64,str,str,u64,f64,f64,f64,f64,f64,f64,f64,f64
9243023,2024-06-07 06:04:59,"""0x8528e8806de8941e02769f797b547368682a8c5184a5db26a122613946f089a0""",2.72,1,1,"""taiko""","""mainnet""",20038042,0.0026175328074991844,0.001512565,8.652629168,5,0.004130097807499185,0.366,125.16570222590961,0.005265922092890303
9243983,2024-06-07 09:16:59,"""0x27fa0a1f1ecf32b9c1cd717300b4b389c73d47881bbd5f97dbd7eb5ae57c7765""",10.32,1,3,"""taiko""","""mainnet""",20039000,0.0035941762960171853,0.000529534,13.574865055,2,0.004123710296017186,0.128,125.16570222590961,0.005265922092890303
9249726,2024-06-08 04:25:35,"""0x13c56f8abb2d3217300e003eca872e097765159f446028171bb145d60c4aa1df""",4.36,1,1,"""taiko""","""mainnet""",20044711,0.0016036889808557471,0.000290989,5.5111670230000005,1,0.001894677980855747,0.154,125.16570222590961,0.005265922092890303
9250779,2024-06-08 07:56:11,"""0xf2547fe0b3023a5f4549b518a3be21aafcde1ab5f170a3a71f74fa5c621ca419""",7.919999999999999,1,1,"""taiko""","""mainnet""",20045756,0.001942934086712906,0.000264791,7.3376137660000005,1,0.002207725086712906,0.12,125.16570222590961,0.005265922092890303
9251635,2024-06-08 10:47:23,"""0x4c1e732ea7dbe622abb74bcf2deb9fa4b7837fced6c978317114843f8f65e6a1""",9.14,1,1,"""taiko""","""mainnet""",20046610,0.001667135101440807,0.000264767,6.296612121000001,1,0.001931902101440807,0.137,125.16570222590961,0.005265922092890303
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9224202,2024-06-04 15:20:47,"""0x7631293fe10fd31930fa930d10600c7b5f3fd09f60daa1dabbe529eec6241c8f""",0.58,1,1,"""taiko""","""mainnet""",20019315,0.0029029678576991073,0.0014576650000000001,9.957596079,5,0.004360632857699108,0.334,125.16570222590961,0.005265922092890303
9228688,2024-06-05 06:17:59,"""0xe2807c1b89bc90621fd7a2a8afeab612c7d8f6b8efed0c6f8e2aa491162b57b6""",25.649999999999995,1,1,"""taiko""","""mainnet""",20023784,0.0017334174782286482,0.0014575450000000002,5.946360072,5,0.0031909624782286484,0.457,125.16570222590961,0.005265922092890303
9232359,2024-06-05 18:32:11,"""0x680072f109714f5a27e26ddaec887eb27d5c95630003298c8ec468c78aa9dc76""",40.42,1,1,"""taiko""","""mainnet""",20027437,0.00573835671067384,0.0029153300000000003,19.683386480000003,10,0.00865368671067384,0.337,125.16570222590961,0.005265922092890303


In [9]:
slot_inclusion_joined_df.filter(pl.col('sequencer_names') == 'base').unique().with_columns(
    pl.col('total_tx_fee_eth').sum().alias('sum'),
    pl.col('total_tx_fee_eth').mean().alias('mean')
)

slot,slot_time,hash,fill_percentage,submission_count,slot_inclusion_rate,sequencer_names,meta_network_name,block_number,base_tx_fee_eth,priority_tx_fee_eth,base_fee_per_gas,priority_fee_gas,total_tx_fee_eth,priority_fee_bid_percent_premium,sum,mean
u32,datetime[ms],str,f64,u32,f64,str,str,u64,f64,f64,f64,f64,f64,f64,f64,f64
9228076,2024-06-05 04:15:35,"""0xdb12c64f825e040c73031313f40a81d8452f65db8a79f568054086f0e57f752e""",99.84000000000002,1,14,"""base""","""mainnet""",20023173,0.000132198724581,0.00008400000000000001,6.295177361,4,0.000216198724581,0.389,2.0928479179761093,0.00035478011832108987
9213595,2024-06-03 03:59:23,"""0xbc2956afc2240fdbf0fca32f607b1175e33a057c17b7d0a626511c06cb3ac78f""",99.91999999999999,1,22,"""base""","""mainnet""",20008764,0.00017026493127000002,0.00016800000000000002,8.107853870000001,8,0.00033826493127000004,0.497,2.0928479179761093,0.00035478011832108987
9246408,2024-06-07 17:21:59,"""0xd07ccd6f39d2e25e9de0cabe064f50e3c85116c82cd69a3f487f33bd4764c640""",99.17,1,9,"""base""","""mainnet""",20041410,0.00042166850497800005,0.000042000000000000004,20.079452618,2,0.00046366850497800004,0.091,2.0928479179761093,0.00035478011832108987
9224899,2024-06-04 17:40:11,"""0x7a7d2efedcd2a704179ba9765668cefebf81dfee553ea7c04bbb8492b71bfb18""",98.03999999999999,1,24,"""base""","""mainnet""",20020009,0.000301714720566,0.00016800000000000002,14.367367646000002,8,0.000469714720566,0.358,2.0928479179761093,0.00035478011832108987
9239987,2024-06-06 19:57:47,"""0x177bdf00696342e29651d776c1b6bf9dd6f0a418c011dd3cb4046f925e4e89c4""",99.93,1,1,"""base""","""mainnet""",20035020,0.00048042242766,0.000042000000000000004,22.87725846,2,0.00052242242766,0.08,2.0928479179761093,0.00035478011832108987
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9234344,2024-06-06 01:09:11,"""0x701cb25c2015dfb1086c5eab69580e782b2f7701fa90b1cbca9b2b9be25aad4f""",98.75,1,1,"""base""","""mainnet""",20029413,0.00031262512186500005,0.000042000000000000004,14.886910565,2,0.00035462512186500004,0.118,2.0928479179761093,0.00035478011832108987
9237188,2024-06-06 10:37:59,"""0x31cb7d4ab6a7c3b9e56bab2ad7a962b247f064221a9a75a60913171475add328""",98.46999999999998,1,1,"""base""","""mainnet""",20032239,0.00028205666823600005,0.000042000000000000004,13.431269916000002,2,0.00032405666823600004,0.13,2.0928479179761093,0.00035478011832108987
9234708,2024-06-06 02:21:59,"""0x8492c9e1b187d10ce383f15cd75e143d578bc40c0dcbd429ea901eb36fd234a4""",99.3,1,1,"""base""","""mainnet""",20029775,0.000240199704864,0.000042000000000000004,11.438081184000001,2,0.000282199704864,0.149,2.0928479179761093,0.00035478011832108987


In [10]:
# VERIFY TAIKO FEE USAGE

In [11]:
slot_inclusion_joined_df.drop_nulls().unique().group_by('sequencer_names').agg(
    pl.col('base_tx_fee_eth').sum().alias('total_base_fees_eth'),
    pl.col('priority_tx_fee_eth').sum().alias('total_priority_fees_eth'),
    pl.col('total_tx_fee_eth').sum().alias('total_eth_fees'),
    pl.col('priority_fee_gas').mean().alias('avg_priority_gas_bid'),
).sort(by='sequencer_names')

sequencer_names,total_base_fees_eth,total_priority_fees_eth,total_eth_fees,avg_priority_gas_bid
str,f64,f64,f64,f64
"""arbitrum""",11.279320519100182,0.9040326159999997,12.183353135100184,1.0045558086560364
"""base""",1.615559917976109,0.4772880000000022,2.092847917976109,3.8528564163417527
"""blast""",0.4956986284629751,0.14697900000000014,0.642677628462975,3.7487948580610606
"""linea""",5.5623157116435396,0.6113755293463412,6.17369124098988,1.3913711816075258
"""metal""",0.152806535727321,0.033641999999999984,0.186448535727321,2.005006257822278
"""optimism""",0.763579147921065,0.1388100000000005,0.9023891479210648,2.288781163434903
"""paradex""",3.3758645145358868,0.028098964800000032,3.4039634793358866,0.09999999999999978
"""scroll""",13.294968835797038,0.10918766425740803,13.40415650005445,0.09704550746164445
"""taiko""",88.73702521420965,36.42867701169988,125.16570222590965,4.981337035634652


### run the dashboard

In [12]:
# # retrieve data from database and store in memory
cached_data = get_data()

# prepare dataframes
slot_inclusion_df = create_slot_inclusion_df(
    cached_data).join(
    pl.from_dict(sequencers_l2),
    left_on="from",
    right_on="sequencer_addresses",
    how="left",
    coalesce=True
).select('slot', 'slot_time', 'hash', 'blob_hashes_length', 'fill_percentage', 'submission_count', 'slot_inclusion_rate', 'sequencer_names', 'meta_network_name')

slot_gas_bidding_df = create_slot_gas_bidding_df(
    cached_data).select('block_number', 'hash', 'base_tx_fee_eth', 'priority_tx_fee_eth', "base_fee_per_gas",
                        "priority_fee_gas", 'total_tx_fee_eth', 'priority_fee_bid_percent_premium')

slot_inclusion_joined_df = slot_inclusion_df.join(
    slot_gas_bidding_df, on="hash", how="left", coalesce=True
)

filtered_data_dict = filter_data_seq(
    sequencers_l2["sequencer_names"], slot_inclusion_joined_df, cached_data
)

sequencer_names_list: list[str] = sorted(sequencers_l2["sequencer_names"])

dashboard = start_interactive_panel(filtered_data_dict, sequencer_names_list)
dashboard.servable()

2024-06-09 is within a day of 2024-06-09


Launching server at http://localhost:44621


Gtk-Message: 16:00:14.664: Failed to load module "canberra-gtk-module"
Gtk-Message: 16:00:14.665: Failed to load module "canberra-gtk-module"


Opening in existing browser session.
